In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

from delta import *

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/app/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b7f37525-2cb3-4e37-b5b3-0db84a1f0fa8;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.4.0/delta-core_2.12-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.4.0!delta-core_2.12.jar (1650ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.4.0/delta-storage-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.4.0!delta-storage.jar (373ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (384ms)
:: resolution report :: resolve 4593ms :: artifacts dl 2416m

In [3]:
df = (
    spark
    .read
    .option('delimiter', ',')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('encoding', 'ISO-8859-1')
    .csv('./dados/demo.csv')
)

In [4]:
df.printSchema()

root
 |-- Nome: string (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Cidade: string (nullable = true)



In [5]:
df.createOrReplaceTempView('demos')

In [6]:
query = """
    SELECT * FROM demos
"""
spark.sql(query).show()

+-----+-----+--------------+
| Nome|Idade|        Cidade|
+-----+-----+--------------+
|  Ana|   28|    SÃ£o Paulo|
|JoÃ£o|   34|Rio de Janeiro|
|Maria|   29|      Curitiba|
+-----+-----+--------------+



In [7]:
df_B = (
    spark
    .read
    .option('delimiter', ',')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .csv('./dados/dados_fake.csv')
).createOrReplaceTempView('fakedata')

In [8]:
query = """
    SELECT * FROM fakedata
"""
spark.sql(query).show()

+--------------------+--------------------+--------------------+------------------+--------------+-------------------+
|       Nome Completo|            Endereço|               Email|Data de Nascimento|           CPF|           Telefone|
+--------------------+--------------------+--------------------+------------------+--------------+-------------------+
|   Sra. Lara Peixoto|Aeroporto Peixoto...| eloah47@hotmail.com|        1954-06-03|583.146.297-82|       81 6153 0345|
| Srta. Heloísa Lopes|Colônia Luiz Feli...|martinsnicole@ig....|        1993-07-23|486.950.712-94|   +55 31 5061 9830|
|   Benjamin Caldeira|Área Nicolas Azev...|ydas-neves@hotmai...|        1962-11-07|549.378.612-55|    (021) 2266 2974|
|   Fernanda Monteiro|Lago da Cunha, 9,...|  dpereira@costa.com|        2009-12-06|142.378.695-55|    (031) 1566 0090|
|       Gabriel Souza|Praia Enzo Castro...|leandro12@yahoo.c...|        2008-07-01|086.571.493-20|       61 2903 0267|
|        Ana Monteiro|Colônia de Carval...|morae

In [ ]:
query = """
    CREATE TABLE tb_fake_people USING DELTA LOCATION '/poetry_delta_spark/delta/'
    AS SELECT
        `Nome Completo` AS nm_nomecompleto,
        `Endereço` AS nm_endereco,
        Email AS nm_email,
        `Data de Nascimento` AS dt_nascimento,
        CPF AS num_cpf,
        Telefone AS num_tel
       FROM fakedata
"""
spark.sql(query)